In [20]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime
import os
import re
import pymysql
import matplotlib.pyplot as plt  
from konlpy.tag import *
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss



In [21]:
import warnings
import urllib.request
warnings.filterwarnings('ignore')
from transformers import logging
logging.set_verbosity_error()

MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [22]:
# pip install torch

In [23]:
# DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
# DATASET_NAME = "finance_data.csv"

In [24]:
# urllib.request.urlretrieve(DATASET_URL, 
#                            filename = DATASET_NAME
#                            )

In [25]:
# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 64

def conver_data(X_data):
    # BERT 입력으로 들어가는 token, mask, segment 저장용 리스트
    tokens, masks, segments = [], [], []
    
    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation=True, padding='max_length', max_length=MAX_SEQ_LEN)
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0] * MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]


In [26]:
df = pd.read_csv('./final news.csv', index_col=0)

# df = pd.DataFrame(result, columns=['stock_id', 'date', 'closing_price', 'market_price', 'high_price', 'low_price'])

df = df.sort_values(ascending=False, by='date_time').reset_index(drop=True)

stock_id = '012450'

df.insert(0, 'stock_id', stock_id)

In [27]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')

In [28]:
df

,stock_id,date_time,open,high,low,close,volume,change
0,012450,2023-07-25 15:20:00,132900.0,133000.0,132800.0,133000.0,1244.0,0.075245
1,012450,2023-07-25 15:19:00,132800.0,132900.0,132700.0,132900.0,726.0,0.150716
2,012450,2023-07-25 15:18:00,132600.0,132800.0,132500.0,132700.0,1515.0,0.075415
3,012450,2023-07-25 15:17:00,132500.0,132600.0,132400.0,132600.0,1975.0,0.075472
4,012450,2023-07-25 15:16:00,132400.0,132500.0,132400.0,132500.0,1141.0,0.075529
...,...,...,...,...,...,...,...,...
194055,012450,2021-07-01 09:05:00,46900.0,46900.0,46850.0,46850.0,923.0,-0.106610
194056,012450,2021-07-01 09:04:00,47000.0,47000.0,46850.0,46900.0,1535.0,-0.212766
194057,012450,2021-07-01 09:03:00,47000.0,47050.0,46900.0,47000.0,2886.0,0.000000
194058,012450,2021-07-01 09:02:00,46900.0,47100.0,46900.0,47000.0,784.0,0.320171


In [29]:
news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','content']
)

In [30]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [31]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=001"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
        
    return news_urls

In [32]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls) :
    news_titles = []
    news_contents =[]
    news_dates = []

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)
        # html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        # news_date = html_date.attrs['data-date-time']
        # news_dates.append(news_date)
    
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [33]:
def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
  pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
  text = re.sub(pattern, '', text)
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
  text = re.sub(pattern, '', text)
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern, '', text)
  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern, '', text)
  pattern = '[^\w\s]'         # 특수기호제거
  text = re.sub(pattern, '', text)
  pattern = '[\n]'            # \n제거
  text = re.sub(pattern, '', text)
  pattern = '[\t]'            # \n제거
  text = re.sub(pattern, '', text)
  pattern = '[\']'           
  text = re.sub(pattern, '', text)
  pattern = '[\"]'            
  text = re.sub(pattern, '', text)
  return text  

In [34]:
def to_dbeaver(NewsDate, NewsTitle, NewsContent) :
    conn = pymysql.connect(
        host='34.64.240.96'
        , user='root'
        , password='tndusWkd1.'
        , db='final_project'
        , charset='utf8'
    )
    cur = conn.cursor()
    
    for date,title,content in zip(NewsDate, NewsTitle, NewsContent):
        sql = "INSERT INTO past_news_realtime VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
        # print(sql)
        try :
            cur.execute(sql)
            conn.commit()
            conn.close()
        except :
            print("에러")
            return
    # conn.commit()
    # conn.close()

In [35]:
# # 입력 데이터(문장) 길이 제한
# MAX_SEQ_LEN = 64

# # token, mask, segment 입력 정의
# token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
# mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
# segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
# bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

# bert_output = bert_outputs[0]

# import tensorflow as tf
# from transformers import BertTokenizer, TFBertForSequenceClassification

# # BERT 모델 불러오기
# MODEL_NAME = "klue/bert-base"
# bert_model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASS, from_pt=True)

# # BERT 모델의 입력 텐서 정의
# input_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids')
# attention_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask')
# token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='token_type_ids')

# # BERT 모델에 입력 텐서 주입하여 출력 얻기
# bert_output = bert_model([input_ids, attention_mask, token_type_ids])[0]


# DROPOUT_RATE = 0.5
# NUM_CLASS = 3
# dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# # Multi-class classification 문제이므로 activation function은 softmax로 설정
# sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
# sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 64

# Multi-class classification 문제이므로 클래스 수 설정
NUM_CLASS = 3

# BERT 모델 불러오기
MODEL_NAME = "klue/bert-base"
bert_model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASS, from_pt=True)

# BERT 모델의 입력 텐서 정의
input_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask')
token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='token_type_ids')

# BERT 모델에 입력 텐서 주입하여 출력 얻기
bert_output = bert_model([input_ids, attention_mask, token_type_ids])[0]

DROPOUT_RATE = 0.5

# Dropout 레이어 추가
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)

# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)

# Sentiment 모델 생성
sentiment_model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=sentiment_layer)

# 모델 컴파일
sentiment_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        metrics=['accuracy'])


In [36]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './best_model.h5'

# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [37]:
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np
import time
import os
import sys
# sys.path.append('./')
import redis_self


# 무한루프 크롤링
while True:
    news_urls = get_top3_news_info()
    a = F_crawling(news_urls)

    # 뉴스가 없는 경우에는 모델링 라벨 처리를 스킵
    if a.empty:
        time.sleep(10)
        continue

    df_all = pd.merge(a, news_df, how='outer', indicator=True)
    news_df = pd.concat([news_df, a], ignore_index=True, keys=['date', 'title', 'content'])
    news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
    str_expr = '_merge == "left_only"'
    df_all2 = df_all.query(str_expr)
    df_all2.drop(columns=['_merge'], inplace=True)

    df_all2['content'] = df_all2['content'].apply(text_clean)
    df_all2['content'] = df_all2['content'].apply(clean_text)
    df_all2['title'] = df_all2['title'].apply(text_clean)
    df_all2['title'] = df_all2['title'].apply(clean_text)
    df_all2['date'] = df_all2['date'].astype(str)

    for i, row in df_all2.iterrows():
        date_str = row['date']
        
        if ':' in df_all2['date'][i]:
            hour_str, minute_str, second_str = df_all2['date'][i].split()[1].split(':')
            df_all2['date'][i] = df_all2['date'][i].split()[0] + ' ' + hour_str.zfill(2) + ':' + minute_str.zfill(2) + ':' + second_str.zfill(2)
            
        if date_str[-8:-6] == '24':
            new_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") + timedelta(days=1)
            df_all2['date'][i] = new_date.strftime('%Y-%m-%d %H:%M:%S')
        elif date_str[-5:-3] == '60':
            new_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") + timedelta(hours=1)
            df_all2['date'][i] = new_date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            hour = int(date_str[-8:-6])
            minute = int(date_str[-5:-3])
            
            df_all2['date'][i] = date_str[:4] + '-' + date_str[5:7] + '-' + date_str[8:10] + ' ' + str(hour).zfill(2) + ':' + str(minute).zfill(2) + ':' + second_str.zfill(2)

    df_all2['date'] = pd.to_datetime(df_all2['date'])

    # 범위 내의 데이터만 처리하고, 범위를 벗어나면 다음 날의 시작 시간으로 날짜 변경
    start_time = pd.to_datetime('09:01:00').time()
    end_time = pd.to_datetime('15:20:00').time()
    next_day_start_time = pd.to_datetime('09:01:00') + pd.DateOffset(days=1)

    mask = (df_all2['date'].dt.time < start_time) | (df_all2['date'].dt.time > end_time)
    df_all2.loc[mask, 'date'] = df_all2.loc[mask, 'date'].apply(lambda x: x + pd.DateOffset(days=1)).dt.strftime('%Y-%m-%d') + ' ' + next_day_start_time.strftime('%H:%M:%S')

    # df_all2 데이터프레임에 뉴스가 없는 경우 스킵하여 다음 루프로 진행
    if df_all2.empty:
        time.sleep(10)
        continue

    df_all2 = df_all2[df_all2['title'] != ' ']
    df_all2 = df_all2[df_all2['content'] != ' ']
    df_all2 = df_all2[df_all2['content'] != '사이트']
    df_all2 = df_all2[df_all2['content'] != ' 사이트 ']
    df_all2 = df_all2[df_all2['title'] != '사이트']
    df_all2 = df_all2[df_all2['date'] != 'None']
    df_all2 = df_all2.dropna(subset=['title'])
    df_all2.drop_duplicates(subset=['title'], inplace=True)
    df_all2.insert(0, 'stock_id', stock_id)

    # # # X_data = [news]  # X_data를 리스트로 감싸서 전달
    input_data = conver_data(df_all2['content'])
    predictions = sentiment_model_best.predict(input_data)
    predicted_label = np.argmax(predictions, axis=1)

    mapping = {1: '1', 0: '0', 2: '-1'}
    # predicted_label을 매핑 딕셔너리를 이용하여 원하는 형식으로 변경
    df_all2['predicted_label'] = pd.Series(predicted_label).replace(mapping).tolist()

    print(df_all2['predicted_label'])

    
#     day = datetime.now()
#     filepath = '/Users/yujin/big16/git/trading_project-1/MUJ/News/The_Relationship_between_news_and_stocks-main/real_time_news_label_{}.csv'.format(day.strftime('%Y%m%d'))
#     if not os.path.exists(filepath):
#         df_all2.to_csv(filepath, index=False, mode='w', encoding='utf-8-sig')
#     else:
#         df_all2.to_csv(filepath, index=False, mode='a', encoding='utf-8-sig', header=False)

    
    # datetime 열을 datetime으로 변환합니다.
    df_all2['date'] = pd.to_datetime(df_all2['date'])
    df_all2['predicted_label'] = df_all2['predicted_label'].astype(float)
    # 날짜별로 그룹화합니다.
    grouped_data = df_all2.groupby(df_all2['date'])

    # 날짜별로 합치고 'predicted_label' 값을 평균으로 계산합니다.
    merged_data = grouped_data.agg({'date': 'first', 'predicted_label': 'mean'}).reset_index(drop=True)

    # 빈 리스트를 생성하여 각 날짜와 라벨을 담을 준비를 합니다.
    NewsDate_list = []
    predicted_label_list = []

    for i, row in merged_data.iterrows():
        # 날짜와 라벨을 리스트에 추가합니다.
        NewsDate_list.append(row['date'].strftime('%Y-%m-%d %H:%M:%S'))
        predicted_label_list.append(str(row['predicted_label']))

    for date, label in zip(NewsDate_list, predicted_label_list):
        mapped_label = mapping.get(label, label)
        print("Mapped Label:", mapped_label)
        redis_self.redis_self.pub_msg('news', (date, mapped_label))

    print(NewsDate_list)
    print(predicted_label_list)
    
    time.sleep(5)
                                             

100%|██████████| 3/3 [00:00<00:00, 5946.56it/s]


1/1 [==============================] - 4s 4s/step
0    0
1    0
2    0
Name: predicted_label, dtype: object
Mapped Label: 0.0
Mapped Label: 0.0
['2023-08-07 12:33:01', '2023-08-07 12:33:05']
['0.0', '0.0']


100%|██████████| 3/3 [00:00<00:00, 5960.64it/s]


1/1 [==============================] - 0s 387ms/step
0    0
1    0
2    0
Name: predicted_label, dtype: object
Mapped Label: 0.0
Mapped Label: 0.0
['2023-08-07 12:34:00', '2023-08-07 12:34:01']
['0.0', '0.0']


100%|██████████| 1/1 [00:00<00:00, 3912.60it/s]


...

100%|██████████| 3/3 [00:00<00:00, 6482.70it/s]


1/1 [==============================] - 0s 354ms/step
0    0
1    0
2    0
Name: predicted_label, dtype: object
Mapped Label: 0.0
Mapped Label: 0.0
Mapped Label: 0.0
['2023-08-07 14:37:05', '2023-08-07 14:37:06', '2023-08-07 14:37:07']
['0.0', '0.0', '0.0']


100%|██████████| 1/1 [00:00<00:00, 3990.77it/s]


1/1 [==============================] - 0s 182ms/step
0    0
Name: predicted_label, dtype: object
Mapped Label: 0.0
['2023-08-07 14:37:15']
['0.0']


ValueError: invalid literal for int() with base 10: ''